# ELYZA-tasks-100 による性能評価

In [ ]:
!pip install accelerate datasets jsonlines sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00


## ELYZA-tasks-100 データセットのダウンロード

モデルのダウンロードには 🤗datasets ライブラリを用いる。

データは `/content/dataset.jsonl` に保存する。

In [ ]:
import json
from pathlib import Path

from datasets import load_dataset

# Load the dataset
ds = load_dataset("elyza/ELYZA-tasks-100")

# Function to convert dataset to JSONL format and print
def dataset_to_jsonl(dataset, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        for entry in dataset:
            # Construct JSON object
            json_obj = {
                "input_text": entry['input'],
                "output_text": entry['output'],
                "eval_aspect": entry['eval_aspect']
            }

            # Write JSON object to file in JSONL format
            json_str = json.dumps(json_obj, ensure_ascii=False)
            file.write(json_str + '\n')

# Convert and write the dataset to a file in JSONL format
path_jsonl = Path("/content/dataset.jsonl")

if (path_jsonl.parent is not None) and (not path_jsonl.parent.exists()):
    path_jsonl.parent.mkdir(parents=True, exist_ok=True)

dataset_to_jsonl(ds["test"], path_jsonl)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

## 評価

ELYZA-tasks-100 データセットを用いて、指定したモデルの評価を実行する。

### プロンプトについて

モデルに回答を生成させるときのプロンプトは、各モデルのモデルカード (🤗HuggingFace の README ページ) に記載されているプロンプトを使用する。

> 📒 **ノート**
>
> この方法は、質問を直接入力する場合や、全モデルで共通のプロンプトを用いる場合に比べて公平性が劣る。
> しかし、実利用時は各モデルが最もよい性能を発揮するように、各モデルのプロンプトをチューニングすることが想定される。
> モデルカード記載のプロンプトをチューニング済みのプロンプトと見なすことで、実利用時の性能を比較することを狙いとする。

### ハイパーパラメータについて

ハイパーパラメータは 🤗HuggingFace の Code Snippet のとおりとする（公開者が指定したハイパーパラメータを、チューン済みとみなす）。
ただし、長文の回答を要求する設問もあるため、`max_new_tokens` は `1024` に統一する。

In [ ]:
import json
import jsonlines
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Setup model and tokenizer
model_name = "Xwin-LM/Xwin-LM-7B-V0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto",
    # trust_remote_code=True, # ⚠️ Microsoft/Phi-1.5 のみ必要
)

def format_prompt(input_text):
    # ✋プロンプトは、モデルに合わせて調整（HuggingFace のモデルカード参照）
    #prompt_template = "<s>[INST] <<SYS>>\nあなたは誠実で優秀な日本人のアシスタントです。ユーザは日本語だけを理解するため、あなたは日本語だけを話します。ユーザの質問や指示に回答してください。\n<</SYS>>\n\n{input} [/INST] "
    (
        prompt_template := "好奇心旺盛なユーザとAIアシスタントが会話しています。"
                "アシスタントは、ユーザの質問に対して、親切・詳細で役立つ回答を提供します。 "
                "USER: {input_text} "
                "ASSISTANT:"
    )
    return prompt_template.format(input_text=input_text)

def generate_text(input_text):
    formatted_prompt = format_prompt(input_text)
    input_ids = tokenizer.encode(
        formatted_prompt,
        add_special_tokens=False,
        return_tensors="pt"
    )

    # Set seed for reproducibility
    seed = 42
    torch.manual_seed(seed)

    # ✋Hparamsは、モデルに合わせて調整（HuggingFace のモデルカード参照）
    tokens = model.generate(
        input_ids.to(device=model.device),
        max_new_tokens=4096,
        temperature=0.7,
    )

    # Remove the input tokens from the generated tokens before decoding
    output_tokens = tokens[0][len(input_ids[0]):]
    return tokenizer.decode(output_tokens, skip_special_tokens=True)

def process_dataset(input_file, output_file):
    # Read dataset and generate predictions
    with jsonlines.open(input_file) as reader, jsonlines.open(output_file, mode='w') as writer:
        for obj in reader:
            prompt = obj['input_text']
            generated_text = generate_text(prompt)
            print(generated_text)
            writer.write({"pred": generated_text})

# Process the dataset
input_dataset = '/content/dataset.jsonl'
output_predictions = '/content/preds.jsonl'
process_dataset(input_dataset, output_predictions)

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/626 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

...

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/7.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

仕事の熱意を取り戻すために、以下の5つのアイデアをお試しください。

1. 目標設定: 短期・長期の目標を設定し、自分に対して達成できるようにサポートしましょう。目標を達成するために必要なステップを考え、定期的に進捗をチェックしましょう。
2. 新しい興味を持つ: 仕事に興味を持つ新しい分野や技術を学びましょう。これにより、仕事に対する関心を刺激し、熱意を取り戻すことができます。
3. チームワークを強化: チームメイトとのコラボレーションを強化し、互いに助け合いましょう。チームワークを通じて、仕事の成果を大切にし、互いに競い合うことで、熱意を取り戻すことができます。
4. 自己成長を目指して: 自分にとって、仕事において成長できる機会を探しましょう。職歴を積み重ねることで、熱意が増すことがあります。
5. 仕事とのバランスを保ちましょう: 仕事と家庭・リラクセーションとのバランスを保ちましょう。仕事の成果を大切にしながら、仕事とのバランスを保ち、熱意を取り戻すことができます。

これらのアイデアを試してみることで、仕事の熱意を取り戻すことができるでしょう。私たちのAIアシスタントは、さらなるアドバイスやサポートをお聞かせできると幸いです。
ある日、クマは好奇心旺盛な日々を過ごしていました。そんな彼は、海辺にある小さな村に訪れたのです。村は美しい海と山に囲まれており、アザラシたちが多く生息していました。

クマは、アザラシと出会い、すぐに友達になりました。彼らは、海の中で遊び、山の中で遊び、そして時々、村の人々とも遊んでいました。クマは、アザラシたちの楽しい生活に感動し、彼らと共に過ごすことを選んだのです。

しかし、彼らが過ごす日々が続くにつれ、クマは家に帰ることを忘れてしまいました。彼の家は遠く、彼はアザラシたちとの別れを覚悟しなければならなかったのです。

そんなある日、クマはアザラシたちと共に、海の奥の世界を探検することを提案しました。彼らは、暗闇と嵐の中で、勇敢に挑戦し、その恐ろしさに打ち勝っていきました。

その後、クマはアザラシたちと共に、山から海へと至る壮大な彼岸を歩いていました。彼らは、旅の途中でさまざまな困難にぶつかりながらも、団結して戦い抜いていました。

ついに、彼らは目の前に家の光景を見つけました。家は美しく、家族たちは嬉しそうに待っていま